In [6]:
import pandas as pd

In [ ]:
def coil(coil_number,stage):
    # load one csv for the coil at the stage B4 or B5
    df = pd.read_csv(f'SignalExport/{coil_number}B{stage}.csv')
    # Turn it into the list and split it by semicolon
    df= list(df.columns)[0].split(";")
    # Finding index for the length and width
    li=df.index("Lengthpoints:") 
    wi=df.index("Values")
    # Slicing the dataframe into 4 lists 
    coil_num =df[0]
    coil_stage = df[1]
    length=df[li+1:wi]   
    width=df[wi+1:-1]
    # Checking the number of observations in the lists
    #print(f"Number of observations:{len(length),len(width)}")
    # Turning length and width measurements from a string into a float 
    length = [float(i) for i in length]
    width = [float(i) for i in width]
    #Duplicate the coil number for the number of observations in the length 
    mult_coil_values= [coil_num for i in range(len(length))]
    mult_coil_stage_values = [coil_stage for i in range(len(length))]
    # Create an empty dataframe 
    my_df = pd.DataFrame()
    # Turn in the lists into the series  
    mult_coil_values = pd.Series(mult_coil_values)
    coil_stage_values= pd.Series(mult_coil_stage_values)
    length_values = pd.Series(length)
    width_values = pd.Series(width)
    # Create columns and fill them with the series 
    my_df['coil_number'] = mult_coil_values
    my_df['coil_stage'] = mult_coil_stage_values
    my_df['length'] = length_values
    my_df['width'] = width_values
    # Drop the zeros
    my_df = my_df[(my_df != 0).all(1)]
    #Looking at only 140-170 m length
    new_df= my_df.loc[(my_df['length'] >= 140) & (my_df['length'] <= 170)]
    #
    new_df["length_m"] = new_df["length"].astype(int)
    # Checking the shape new_df_B4.shape 
    #print(f"The shape of the sliced dataset is {new_df.shape}")
    return new_df

In [ ]:
# Defining the coil
coil_number = 10080
stage = 4

In [ ]:
def calculating_difference(B4_df = coil(coil_number,stage),B5_df = coil(coil_number,stage+1)):
    # Estimating average width at each meter
    average_B4 = pd.Series(B4_df.groupby('length_m')['width'].mean())
    average_B5 = pd.Series(B5_df.groupby('length_m')['width'].mean())
    # Creating a dataframe where we see the averages of B4 and B5 and the differences 
    two_df = pd.DataFrame()
    two_df['Average_B4'],two_df['Average_B5'] =[average_B4, average_B5]
    two_df["difference"] = two_df["Average_B5"] - two_df["Average_B4"]
    two_df = two_df.rename_axis('Length').reset_index()
    display(two_df.head())